In [62]:
import numpy as np
import pandas as pd

#CentrinManualScore CSV location of intephase labels
fp = r'\\allen\aics\assay-dev\MicroscopyOtherData\Ruian\Projects\aicsassaydevknowledgetransfer\interphase_label\3500001603_all_cell_scores_mitosislabel_interphaselabel.csv'

df = pd.read_csv(fp)
print(df.shape)
df.head(2)

(610, 30)


,Index,Version,inputFolder,inputFilename,imageXyPixelSize,imageZPixelSize,plate_ID,well_ID,position_ID,colony_position,...,outputThisCellIndex,Labeler1,Score1,Labeler2,Score2,FinalScore,MitosisLabel,InterphaseLabel,GoodNucBadCell,Unnamed: 29
0,1,3,//allen/aics/microscopy/PRODUCTION/PIPELINE_4_...,3500001603_100X_20171222_1-Scene-01-P1-E04.czi,0.108,0.29,3500001603,E04,P01,NaN,...,1,Ruian,-1,Sara,-1,-1,5,Nan,0,NaN
1,2,3,//allen/aics/microscopy/PRODUCTION/PIPELINE_4_...,3500001603_100X_20171222_1-Scene-01-P1-E04.czi,0.108,0.29,3500001603,E04,P01,NaN,...,2,Ruian,1,Sara,1,1,7,Nan,0,NaN


In [64]:
# QUILT MANIFEST 
fp1 = r"\\allen\aics\assay-dev\users\Frick\PythonProjects\WHEREdoes_this_go\metadata.csv"
df1 = pd.read_csv(fp1)

In [69]:
#refine quilt manifest to centrin, and correct plate id and the well_ids from CentrinManualScore csv

#first get the list of wells and list of plate ids from CentrinManualScore CSV
well_list = df['well_ID'].unique()
plate_list = df['plate_ID'].unique()
print(well_list)
print(plate_list)

dfc = df1[df1['Protein'].str.contains('Centrin')] #choose centrin
well_log = np.asarray([True if x[0]+'0'+x[1] in well_list else False for x in dfc['WellName'].tolist()])
well_log = np.asarray([True if x in plate_list else False for x in dfc['PlateId'].tolist()])
dfcp = dfc[well_log]
dfcp.head(1)

['E04' 'E06' 'E07' 'E08' 'F04' 'F05' 'F06' 'F07' 'F08']
[3500001603]


,CellId,CellIndex,CellLine,CellLineId,CellLineId/Name,CellPopulationId,ChannelNumber405,ChannelNumber638,ChannelNumberBrightfield,ChannelNumberStruct,...,ch_memb,ch_seg_cell,ch_seg_nuc,ch_struct,ch_trans,save_feats_path,save_reg_path,save_reg_path_flat,save_reg_path_flat_proj,FeatureExplorerURL
12902,62651,1,AICS-32,35,AICS-32,5379,5,1,6,3,...,3,1,0,4,5,cell_features/7b8b5c74_9938_62651_feats.json,cell_images_3d/5a810e7c_9938_62651_reg.tiff,cell_images_2d/c58fdb7d_9938_62651_reg_flat.png,cell_images_2d_projections/94e912f9_9938_62651...,https://cfe.allencell.org/?cellSelectedFor3D=6...


In [75]:
#now check per well
#CentrinManualScore csv
dfkeep = df[df['FinalScore']==1]
for plate,group in dfkeep.groupby('plate_ID'):
    for well,g in group.groupby('well_ID'):
        dfg = g.groupby(['position_ID']).first()
        print(well,"number of FOVS",dfg.shape[0],'//',"number of cells",g.shape[0])
        print(dfg['outputThisCellIndex'].to_list())

E04 number of FOVS 5 // number of cells 23
[2, 1, 3, 2, 6]
E06 number of FOVS 6 // number of cells 28
[1, 4, 1, 2, 2, 1]
E07 number of FOVS 7 // number of cells 19
[1, 4, 1, 11, 2, 1, 2]
E08 number of FOVS 4 // number of cells 13
[6, 5, 4, 4]
F04 number of FOVS 7 // number of cells 47
[1, 1, 1, 3, 2, 1, 1]
F05 number of FOVS 8 // number of cells 32
[5, 6, 6, 1, 1, 1, 1, 1]
F06 number of FOVS 3 // number of cells 11
[1, 2, 5]
F07 number of FOVS 5 // number of cells 18
[1, 5, 8, 4, 8]
F08 number of FOVS 1 // number of cells 3
[2]


In [76]:
#manifest csv
for plate,group in dfcp.groupby('PlateId'):
    for well,g in group.groupby('WellName'):
        dfg = g.groupby(['FOVId']).first()
        print(well,"number of FOVS",dfg.shape[0],'//',"number of cells",g.shape[0])
        print(dfg['CellIndex'].to_list())
       

E4 number of FOVS 5 // number of cells 26
[2, 1, 5, 6, 3]
E6 number of FOVS 6 // number of cells 25
[2, 1, 1, 1, 2, 1]
E7 number of FOVS 6 // number of cells 22
[1, 2, 1, 11, 2, 1]
E8 number of FOVS 4 // number of cells 16
[2, 1, 3, 2]
F4 number of FOVS 7 // number of cells 44
[1, 2, 1, 1, 3, 1, 1]
F5 number of FOVS 8 // number of cells 23
[10, 6, 1, 1, 1, 2, 1, 1]
F6 number of FOVS 3 // number of cells 7
[1, 5, 2]
F7 number of FOVS 5 // number of cells 15
[3, 7, 3, 7, 1]
F8 number of FOVS 1 // number of cells 2
[1]


In [ ]:
#playground that is not useful anymore -- > see below

In [36]:
for plate,group in df.groupby('plate_ID'):
    print(plate)
    dfg = group.groupby(['well_ID','position_ID']).first()
    print("number of FOVS",dfg.shape)
    dfg.head()
    print(group['well_ID'].unique())
    well_list = group['well_ID'].unique()

3500001603
number of FOVS (56, 28)
['E04' 'E06' 'E07' 'E08' 'F04' 'F05' 'F06' 'F07' 'F08']


In [20]:
dfc = df1[df1['Protein'].str.contains('Centrin')]
dfcg = dfc.groupby('FOVId').first()
dfcg.head(5)

,CellId,CellIndex,CellLine,CellLineId,CellLineId/Name,CellPopulationId,ChannelNumber405,ChannelNumber638,ChannelNumberBrightfield,ChannelNumberStruct,...,ch_memb,ch_seg_cell,ch_seg_nuc,ch_struct,ch_trans,save_feats_path,save_reg_path,save_reg_path_flat,save_reg_path_flat_proj,FeatureExplorerURL
FOVId,,,,,,,,,,,,,,,,,,,,,
9459,68343,2,AICS-32,35,AICS-32,5375,5,1,6,3,...,3,1,0,4,5,cell_features/0e10e99d_9459_68343_feats.json,cell_images_3d/25a06fd7_9459_68343_reg.tiff,cell_images_2d/4c67b992_9459_68343_reg_flat.png,cell_images_2d_projections/04b4bb85_9459_68343...,https://cfe.allencell.org/?cellSelectedFor3D=6...
9462,68504,4,AICS-32,35,AICS-32,5375,5,1,6,3,...,3,1,0,4,5,cell_features/336777f9_9462_68504_feats.json,cell_images_3d/dca93d71_9462_68504_reg.tiff,cell_images_2d/d86bea9b_9462_68504_reg_flat.png,cell_images_2d_projections/46e57b02_9462_68504...,https://cfe.allencell.org/?cellSelectedFor3D=6...
9468,68400,1,AICS-32,35,AICS-32,5375,5,1,6,3,...,3,1,0,4,5,cell_features/14236a17_9468_68400_feats.json,cell_images_3d/88cf2ec2_9468_68400_reg.tiff,cell_images_2d/96b1e799_9468_68400_reg_flat.png,cell_images_2d_projections/28b68ead_9468_68400...,https://cfe.allencell.org/?cellSelectedFor3D=6...
9471,68598,2,AICS-32,35,AICS-32,5375,5,1,6,3,...,3,1,0,4,5,cell_features/b9b25a8f_9471_68598_feats.json,cell_images_3d/722e9577_9471_68598_reg.tiff,cell_images_2d/df572fea_9471_68598_reg_flat.png,cell_images_2d_projections/49be75a7_9471_68598...,https://cfe.allencell.org/?cellSelectedFor3D=6...
9482,68552,8,AICS-32,35,AICS-32,5375,5,1,6,3,...,3,1,0,4,5,cell_features/06668547_9482_68552_feats.json,cell_images_3d/3c869bea_9482_68552_reg.tiff,cell_images_2d/a46a8a26_9482_68552_reg_flat.png,cell_images_2d_projections/820efa20_9482_68552...,https://cfe.allencell.org/?cellSelectedFor3D=6...


In [24]:
for plate,group in dfc.groupby('PlateId'):
    print(plate)
    dfg = group.groupby(['FOVId']).first()
    print("number of FOVS",dfg.shape)
    dfg.head()

3500001569
number of FOVS (24, 55)
3500001571
number of FOVS (22, 55)
3500001581
number of FOVS (19, 55)
3500001582
number of FOVS (23, 55)
3500001588
number of FOVS (32, 55)
3500001590
number of FOVS (23, 55)
3500001597
number of FOVS (16, 55)
3500001598
number of FOVS (26, 55)
3500001603
number of FOVS (45, 55)
3500001607
number of FOVS (17, 55)
3500001610
number of FOVS (30, 55)
3500001636
number of FOVS (7, 55)
3500001637
number of FOVS (8, 55)
3500001656
number of FOVS (26, 55)
3500001657
number of FOVS (33, 55)
9999999104
number of FOVS (71, 55)
9999999107
number of FOVS (70, 55)


In [41]:
import numpy as np
dfcp0 = dfc[dfc['PlateId'] == 3500001603]
well_log = np.asarray([True if x[0]+'0'+x[1] in well_list else False for x in dfcp0['WellName'].tolist()])
dfcp = dfcp0[well_log]
print(dfcp.shape)

for fovid,group in dfcp.groupby('FOVId'):
    print(fovid,np.unique(group['WellName']),group.shape)

(180, 56)
9911 ['E6'] (5, 56)
9913 ['F4'] (6, 56)
9915 ['E8'] (8, 56)
9917 ['F5'] (1, 56)
9918 ['F4'] (8, 56)
9919 ['E7'] (3, 56)
9920 ['E4'] (12, 56)
9921 ['E6'] (5, 56)
9922 ['E7'] (3, 56)
9923 ['E6'] (2, 56)
9924 ['E7'] (9, 56)
9925 ['E8'] (4, 56)
9926 ['F5'] (1, 56)
9928 ['F4'] (7, 56)
9929 ['E6'] (3, 56)
9930 ['F4'] (4, 56)
9931 ['F5'] (2, 56)
9932 ['F7'] (7, 56)
9934 ['F4'] (2, 56)
9936 ['F6'] (3, 56)
9938 ['E4'] (3, 56)
9941 ['F5'] (3, 56)
9944 ['E6'] (3, 56)
9945 ['E7'] (2, 56)
9946 ['E7'] (2, 56)
9947 ['F5'] (4, 56)
9949 ['E7'] (3, 56)
9950 ['F5'] (2, 56)
9952 ['F7'] (1, 56)
9954 ['F7'] (3, 56)
9955 ['E4'] (2, 56)
9958 ['F7'] (2, 56)
9959 ['F4'] (2, 56)
9961 ['F6'] (2, 56)
9962 ['F5'] (7, 56)
9964 ['F4'] (15, 56)
9965 ['E6'] (7, 56)
9966 ['E4'] (2, 56)
9967 ['F7'] (2, 56)
9971 ['F5'] (3, 56)
9972 ['E4'] (7, 56)
9974 ['E8'] (2, 56)
9975 ['F6'] (2, 56)
9977 ['F8'] (2, 56)
9978 ['E8'] (2, 56)


['E4' 'E6' 'E7' 'E8' 'F4' 'F5' 'F6' 'F7' 'F8']


array(['E04', 'E06', 'E07', 'E08', 'F04', 'F05', 'F06', 'F07', 'F08'],
      dtype=object)

In [52]:
#CentrinManualScore csv
dfkeep = df[df['FinalScore']==1]
for plate,group in dfkeep.groupby('plate_ID'):
    print(plate)
    dfg = group.groupby(['well_ID','position_ID']).first()
    print("number of FOVS",dfg.shape[0])
    print("number of cells",group.shape[0])
    dfg.head()
    print(group['well_ID'].unique())

3500001603
number of FOVS 46
number of cells 194
['E04' 'E06' 'E07' 'E08' 'F04' 'F05' 'F06' 'F07' 'F08']


In [54]:
#manifest csv
for plate,group in dfcp.groupby('PlateId'):
    print(plate)
    dfg = group.groupby(['FOVId']).first()
    print("number of FOVS",dfg.shape[0])
    print("number of cells",group.shape[0])
    dfg.head()
    print(group['WellName'].unique())

3500001603
number of FOVS 45
number of cells 180
['E4' 'E6' 'E7' 'E8' 'F4' 'F5' 'F6' 'F7' 'F8']


In [55]:
dfcp.head()

,CellId,CellIndex,CellLine,CellLineId,CellLineId/Name,CellPopulationId,ChannelNumber405,ChannelNumber638,ChannelNumberBrightfield,ChannelNumberStruct,...,ch_memb,ch_seg_cell,ch_seg_nuc,ch_struct,ch_trans,save_feats_path,save_reg_path,save_reg_path_flat,save_reg_path_flat_proj,FeatureExplorerURL
12902,62651,1,AICS-32,35,AICS-32,5379,5,1,6,3,...,3,1,0,4,5,cell_features/7b8b5c74_9938_62651_feats.json,cell_images_3d/5a810e7c_9938_62651_reg.tiff,cell_images_2d/c58fdb7d_9938_62651_reg_flat.png,cell_images_2d_projections/94e912f9_9938_62651...,https://cfe.allencell.org/?cellSelectedFor3D=6...
12903,62653,3,AICS-32,35,AICS-32,5379,5,1,6,3,...,3,1,0,4,5,cell_features/8ec1d95c_9938_62653_feats.json,cell_images_3d/7b524721_9938_62653_reg.tiff,cell_images_2d/5d2b68b4_9938_62653_reg_flat.png,cell_images_2d_projections/4f4712a6_9938_62653...,https://cfe.allencell.org/?cellSelectedFor3D=6...
12904,62655,5,AICS-32,35,AICS-32,5379,5,1,6,3,...,3,1,0,4,5,cell_features/a2ecb3b2_9938_62655_feats.json,cell_images_3d/8c3daf97_9938_62655_reg.tiff,cell_images_2d/c2a6fc55_9938_62655_reg_flat.png,cell_images_2d_projections/be99f83e_9938_62655...,https://cfe.allencell.org/?cellSelectedFor3D=6...
12905,62672,2,AICS-32,35,AICS-32,5379,5,1,6,3,...,3,1,0,4,5,cell_features/93167eb2_9920_62672_feats.json,cell_images_3d/01e1a197_9920_62672_reg.tiff,cell_images_2d/0e2b8063_9920_62672_reg_flat.png,cell_images_2d_projections/58c13871_9920_62672...,https://cfe.allencell.org/?cellSelectedFor3D=6...
12906,62674,4,AICS-32,35,AICS-32,5379,5,1,6,3,...,3,1,0,4,5,cell_features/91f0a7d6_9920_62674_feats.json,cell_images_3d/8223ff50_9920_62674_reg.tiff,cell_images_2d/49793d56_9920_62674_reg_flat.png,cell_images_2d_projections/107f1c23_9920_62674...,https://cfe.allencell.org/?cellSelectedFor3D=6...


In [61]:
#now check per well
#CentrinManualScore csv
dfkeep = df[df['FinalScore']==1]
for plate,group in dfkeep.groupby('plate_ID'):
    for well,g in group.groupby('well_ID'):
        dfg = g.groupby(['position_ID']).first()
        print(well,"number of FOVS",dfg.shape[0],'//',"number of cells",g.shape[0])

E04 number of FOVS 5 // number of cells 23
E06 number of FOVS 6 // number of cells 28
E07 number of FOVS 7 // number of cells 19
E08 number of FOVS 4 // number of cells 13
F04 number of FOVS 7 // number of cells 47
F05 number of FOVS 8 // number of cells 32
F06 number of FOVS 3 // number of cells 11
F07 number of FOVS 5 // number of cells 18
F08 number of FOVS 1 // number of cells 3


In [60]:
#manifest csv
for plate,group in dfcp.groupby('PlateId'):
    for well,g in group.groupby('WellName'):
        dfg = g.groupby(['FOVId']).first()
        print(well,"number of FOVS",dfg.shape[0],'//',"number of cells",g.shape[0])
       

E4 number of FOVS 5 // number of cells 26
E6 number of FOVS 6 // number of cells 25
E7 number of FOVS 6 // number of cells 22
E8 number of FOVS 4 // number of cells 16
F4 number of FOVS 7 // number of cells 44
F5 number of FOVS 8 // number of cells 23
F6 number of FOVS 3 // number of cells 7
F7 number of FOVS 5 // number of cells 15
F8 number of FOVS 1 // number of cells 2
